### Basic library imports

In [1]:
import re
import os
import pandas as pd
import pytesseract
from PIL import Image

### Read Dataset

In [2]:
DATASET_FOLDER = 'D:/Download/Gunjit_IIT BHU Downloads/Amazon ML Challnge/student_resource 3/dataset/'
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

### Run Sanity check using src/sanity.py

In [3]:
!python sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out.csv

Parsing successfull for file: ../dataset/sample_test_out.csv


In [4]:
!python sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out_fail.csv

Error: Invalid unit [lbs] found in 6.75 lbs. Allowed units: {'volt', 'fluid ounce', 'cup', 'pint', 'inch', 'kilowatt', 'pound', 'litre', 'kilogram', 'kilovolt', 'yard', 'cubic inch', 'ounce', 'millilitre', 'decilitre', 'milligram', 'ton', 'millivolt', 'watt', 'centimetre', 'cubic foot', 'millimetre', 'foot', 'microlitre', 'imperial gallon', 'gram', 'gallon', 'quart', 'centilitre', 'microgram', 'metre'}


### Download images

In [ ]:
from utils import download_images
download_images(sample_test['image_link'], 'D:/Download/Gunjit_IIT BHU Downloads/Amazon ML Challnge/student_resource 3/images/')

In [3]:
assert len(os.listdir('D:/Download/Gunjit_IIT BHU Downloads/Amazon ML Challnge/student_resource 3/images/')) > 0

In [4]:
sample_test

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/41-NCxNuBx...,658003,width
1,1,https://m.media-amazon.com/images/I/41-NCxNuBx...,658003,depth
2,2,https://m.media-amazon.com/images/I/417NJrPEk+...,939426,maximum_weight_recommendation
3,3,https://m.media-amazon.com/images/I/417SThj+Sr...,276700,voltage
4,4,https://m.media-amazon.com/images/I/417SThj+Sr...,276700,wattage
...,...,...,...,...
83,83,https://m.media-amazon.com/images/I/81IYdOV0mV...,721522,maximum_weight_recommendation
84,84,https://m.media-amazon.com/images/I/81PG3ea0MO...,240413,voltage
85,85,https://m.media-amazon.com/images/I/81aZ2ozp1G...,805279,maximum_weight_recommendation
86,86,https://m.media-amazon.com/images/I/81qUmRUUTT...,603688,maximum_weight_recommendation


In [5]:
import os
import pandas as pd
from PIL import Image
import pytesseract
import re
from src.utils import download_images
from src.constants import ALLOWED_UNITS

# Function to extract text from an image
def extract_text_from_image(image_path):
    try:
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

# Function to extract entity value from text
def extract_entity_value(text, entity_name):
    float_pattern = r'(\d+(\.\d+)?)'
    units_pattern = '|'.join(ALLOWED_UNITS[entity_name])
    
    match = re.search(f'{float_pattern} ({units_pattern})', text)
    if match:
        return f"{match.group(1)} {match.group(3)}"
    return ""

# Function to get image file name from image link
def get_image_filename(image_link):
    return os.path.basename(image_link)

# Generate predictions
def generate_predictions(sample_test):
    predictions = []
    for _, row in sample_test.iterrows():
        image_link = row['image_link']
        image_name = get_image_filename(image_link)  # Extract file name from image link
        image_path = f"images/test/{image_name}"
        
        # Extract text from image
        text = extract_text_from_image(image_path)
        
        # Extract entity value from text
        entity_value = extract_entity_value(text, row['entity_name'])
        
        # Append the result
        predictions.append([row['index'], entity_value])
    
    return predictions

# Download test images (use provided function)
download_images(sample_test["image_link"], "images/test/")

# Generate predictions for test data
predictions = generate_predictions(sample_test)

# Convert to DataFrame and save as CSV
output_df = pd.DataFrame(predictions, columns=['index', 'prediction'])
output_df.to_csv("output.csv", index=False)

# Sanity check
os.system('python src/sanity.py --file output.csv')


ModuleNotFoundError: No module named 'constants'

In [ ]:
from PIL import Image
import pytesseract

# Function to extract text from an image
def extract_text_from_image(image_path):
    try:
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""


In [ ]:
import re

def extract_entity_value(text, entity_name):
    # Define the patterns to capture float numbers and units
    float_pattern = r'(\d+(\.\d+)?)'
    
    # Use constants.py to load the allowed units
    from src.constants import ALLOWED_UNITS  # Assuming ALLOWED_UNITS is a dict mapping entity names to valid units
    
    units_pattern = '|'.join(ALLOWED_UNITS[entity_name])
    
    # Find the first match of number and unit in the text
    match = re.search(f'{float_pattern} ({units_pattern})', text)
    if match:
        return f"{match.group(1)} {match.group(3)}"
    return ""


In [ ]:
def generate_predictions(test_df):
    predictions = []
    for idx, row in test_df.iterrows():
        image_path = f"images/test/{idx}.jpg"
        text = extract_text_from_image(image_path)
        entity_value = extract_entity_value(text, row['entity_name'])
        predictions.append([row['index'], entity_value])
    return predictions

# Generate predictions for test data
predictions = generate_predictions(sample_test)

# Convert to DataFrame and save as CSV
output_df = pd.DataFrame(predictions, columns=['index', 'prediction'])
output_df.to_csv("output.csv", index=False)
